In [1]:
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/My Drive/Dissertation/envs/point_fall.py" .

Mounted at /content/drive/


In [2]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_fall_pheromone/{name}" 

    torch.save({
      'meta_controller': model.pheromone_paths,
      'controller': {
          'critic': model.controller.critic.state_dict(),
          'actor': model.controller.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_fall_pheromone/{name}" 
    checkpoint = torch.load(path)

    model.pheromone_paths = copy.deepcopy(checkpoint['meta_controller'])

    model.controller.critic.load_state_dict(checkpoint['controller']['critic'])
    model.controller.critic_target = copy.deepcopy(model.controller.critic)
    model.controller.actor.load_state_dict(checkpoint['controller']['actor'])
    model.controller.actor_target = copy.deepcopy(model.controller.actor)

    # model.eval() for evaluation instead
    model.eval()
    model.controller.eval()

In [3]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [4]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [5]:
from point_fall import PointFallEnv 
env = NormalizedEnv(PointFallEnv(4))

***

In [6]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [7]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [8]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
  
# (state, action) -> (next_state, reward, done)
transition_meta = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done', 'state_seq', 'action_seq'))

# replay memory D with capacity N
class ReplayMemoryMeta(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition_meta(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [9]:
DEPTH = 128

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, DEPTH)
        self.fc2 = nn.Linear(DEPTH, DEPTH)
        self.head = nn.Linear(DEPTH, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, DEPTH)
        self.l2 = nn.Linear(DEPTH, DEPTH)
        self.l3 = nn.Linear(DEPTH, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, DEPTH)
        self.l5 = nn.Linear(DEPTH, DEPTH)
        self.l6 = nn.Linear(DEPTH, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [10]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions, is_meta=False):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        self.is_meta = is_meta

        #Create replay buffer
        self.memory = ReplayMemory(100000) if not self.is_meta else ReplayMemoryMeta(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 20000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self, off_policy_correction=None):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)

        if not self.is_meta:
            batch = transition(*zip(*transitions))
            action_batch = torch.cat(batch.action)
        else:
            batch = transition_meta(*zip(*transitions))

            action_batch = torch.cat(batch.action)
            state_seq_batch = torch.stack(batch.state_seq)
            action_seq_batch = torch.stack(batch.action_seq)

            action_batch = off_policy_correction(action_batch.cpu().numpy(), state_seq_batch.cpu().numpy(), action_seq_batch.cpu().numpy())
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # print losses
            #if self.total_it % (50 * 50 if self.is_meta else 500 * 50) == 0:
            #    print(f"{self.is_meta} controller;\n\tcritic loss: {critic_loss.item()}\n\tactor loss: {actor_loss.item()}")

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, 2 * self.tau / 5)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup, decay_epsilon):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

In [11]:
from operator import itemgetter
class HIRO(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(HIRO, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        self.goal_dim = [0, 1]
        self.goal_dimen = 2
      
        # a list of tuple of form (reward, path); keep top 5
        self.pheromone_paths = []

        self.controller = TD3(nb_states + len(self.goal_dim), nb_actions).to(device)
        #self.controller.depsilon = 1.0 / 10000

    def add_path(self, reward, path):
        # prefer higher-reward paths
        self.pheromone_paths.append((reward, path))
        self.pheromone_paths.sort(key=itemgetter(0), reverse=True)
        self.pheromone_paths = self.pheromone_paths[:5] # only keep top 5

    def teach_controller(self):
        self.controller.update_policy()

    def h(self, state, goal, next_state):
        #return goal
        return state[:,self.goal_dim] + goal - next_state[:,self.goal_dim]
    #def intrinsic_reward(self, action, goal):
    #    return torch.tensor(1.0 if self.goal_reached(action, goal) else 0.0, device=device) 
    #def goal_reached(self, action, goal, threshold = 0.1):
    #    return torch.abs(action - goal) <= threshold
    def intrinsic_reward(self, reward, state, goal, next_state):
        #return torch.tensor(2 * reward if self.goal_reached(state, goal, next_state) else reward / 10, device=device) #reward / 2
        # just L2 norm
        return -torch.pow(sum(torch.pow(state.squeeze(0)[self.goal_dim] + goal.squeeze(0) - next_state.squeeze(0)[self.goal_dim], 2)), 0.5)
    def goal_reached(self, state, goal, next_state, threshold = 0.1):
        return torch.pow(sum(torch.pow(state.squeeze(0)[self.goal_dim] + goal.squeeze(0) - next_state.squeeze(0)[self.goal_dim], 2)), 0.5) <= threshold
        #return torch.pow(sum(goal.squeeze(0), 2), 0.5) <= threshold

    def observe_controller(self, s_t, a_t, s_t1, r_t, done):
        self.controller.memory.store(s_t, a_t, s_t1, r_t, done)

    def select_goal(self, s_t, warmup, is_training):
        if warmup or len(self.pheromone_paths) == 0:
            return torch.tensor([np.random.uniform(-1.,1.,len(self.goal_dim))], device=device, dtype=torch.float)
        
        time_index = 3
        #cur_t = s_t.squeeze(0)[time_index] # time
        cur_pos = s_t.squeeze(0)[self.goal_dim]

        goal = torch.tensor([0] * len(self.goal_dim), device=device, dtype=torch.float)

        min_rew = -60 # min(self.pheromone_paths, key = lambda t: t[0])[0]
        tot_rew = sum([t[0] for t in self.pheromone_paths]) - len(self.pheromone_paths) * min_rew

        for rew, path in self.pheromone_paths:
            breakdown = tuple(map(torch.stack, zip(*path)))
            positions = torch.stack([breakdown[i] for i in self.goal_dim], axis=-1)
            chosen_i = torch.argmin(torch.pow(torch.sum(torch.pow(positions - cur_pos, 2), axis=1), 0.5))
            
            # assume c = 10
            # basically, in chosen path, go 10 steps ahead from position closest
            # to the currently observed one
            chosen_point = path[min(chosen_i + 10, len(path) - 1)]

            #chosen_point = path[torch.argmin(torch.abs(breakdown[time_index] - cur_t))]
            goal += (rew - min_rew) * chosen_point[self.goal_dim]
        
        goal /= tot_rew
        goal = goal - s_t.squeeze(0)[self.goal_dim] # make goal relative to given position

        return goal.unsqueeze(0)
    
    def select_action(self, s_t, g_t, warmup, decay_epsilon):
        sg_t = torch.cat([s_t, g_t], 1).float()
        return self.controller.select_action(sg_t, warmup, decay_epsilon)

In [12]:
import time
SAVE_OFFSET = 6
def train_model():
    global SAVE_OFFSET
    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = HIRO(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    observation = None
    
    warmup = 200
    num_episodes = 4000 # M
    episode_durations = []
    goal_durations = []

    steps = 0
    c = 10

    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        overall_intrinsic = 0
        episode_steps = 0
        done = False
        goals_done = 0

        state_seq = None

        while not done:
            goal = agent.select_goal(state, i_episode <= warmup, True)
            #goal_durations.append((steps, goal[:,0]))

            first_goal = goal
            goal_done = False
            total_extrinsic = 0

            while not done and not goal_done:
                joint_goal_state = torch.cat([state, goal], axis=1).float()

                # agent pick action ...
                action = agent.select_action(state, goal, i_episode <= warmup, True)
                
                # env response with next_observation, reward, terminate_info
                observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                steps += 1
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                next_goal = agent.h(state, goal, next_state)
                joint_next_state = torch.cat([next_state, next_goal], axis=1).float()
                
                if max_episode_length and episode_steps >= max_episode_length -1:
                    done = True
                    
                extrinsic_reward = torch.tensor([reward], device=device)
                intrinsic_reward = agent.intrinsic_reward(reward, state, goal, next_state).unsqueeze(0)
                #intrinsic_reward = agent.intrinsic_reward(action, goal).unsqueeze(0)

                overall_reward += reward
                total_extrinsic += reward
                overall_intrinsic += intrinsic_reward

                goal_reached = agent.goal_reached(state, goal, next_state)
                #goal_done = agent.goal_reached(action, goal)

                # agent observe and update policy
                agent.observe_controller(joint_goal_state, action, joint_next_state, intrinsic_reward, done) #goal_done.item())

                if state_seq is None:
                    state_seq = state
                else:
                    state_seq = torch.cat([state_seq, state])

                episode_steps += 1

                if goal_reached:
                    goals_done += 1
                
                if (episode_steps % c) == 0:
                    goal_done = True

                state = next_state
                goal = next_goal
                
                if i_episode > warmup:
                    agent.teach_controller()

        # once episode finishes, append full path to manager
        agent.add_path(overall_reward, state_seq)

        goal_durations.append((i_episode, overall_intrinsic / episode_steps))
        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, goal_durations)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0:
                print(f"{i_episode}: {np.mean(dur[-100:])}")
            if i_episode >= 400 and i_episode % 100 == 0 and np.mean(dur[-100:]) <= -49.0:
                print(f"Unlucky after {i_episode} eps! Terminating...")
                return None
            if np.mean(dur[-100:]) >= 90:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"hiro_fall_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent

    return None # did not train

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def eval_model(agent, episode_durations, goal_attack, action_attack, same_noise):
    agent.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for l2norm in np.arange(0.0,0.51,0.05):
        
        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)

            if goal_attack:
                g_state = g_state + state_range * noise
                g_state = torch.max(torch.min(g_state, state_max), state_min).float()
            if action_attack:
                if same_noise:
                    state = state + state_range * noise
                else:
                    state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal = agent.select_goal(g_state, False, False)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                    if goal_attack:
                        g_next_state = g_next_state + state_range * noise
                        g_next_state = torch.max(torch.min(g_next_state, state_max), state_min).float()
                    if action_attack:
                        if same_noise:
                            next_state = next_state + state_range * noise
                        else:
                            next_state = next_state + state_range * torch.FloatTensor(next_state.shape).uniform_(-l2norm, l2norm).to(device)
                        next_state = torch.max(torch.min(next_state, state_max), state_min).float()

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(l2norm, 2)].append(overall_reward / num_episodes)

In [13]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data - eps * sign_data_grad * state_range

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_action(state, goal, agent, eps, target, targeted):
    #state = torch.tensor(state, requires_grad=True)
    state = state.clone().detach().requires_grad_(True)
    goal = goal.clone().detach()

    sg_t = torch.cat([state, goal], 1).float()

    if targeted:
        # initial forward pass
        action = agent.controller.actor(sg_t)
        action = torch.clamp(action, -1., 1.)

        loss = F.mse_loss(action, target)
    else:
        loss = agent.controller.critic.Q1(sg_t, agent.controller.actor(sg_t)).mean()

    agent.controller.actor.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return state_p

def apply_fgsm(agent, episode_durations, targeted):
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for eps in np.arange(0.0, 0.201, 0.02):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            og_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            goal = agent.select_goal(og_state, False, False)
            state = fgsm_action(og_state, goal, agent, eps, TARGET_ACTION, targeted)

            episode_steps = 0
            done = False
            while not done:
                goal = agent.select_goal(state, False, False)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())

                    next_og_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    goal_temp = agent.h(state, goal, next_og_state)
                    next_state = fgsm_action(next_og_state, goal_temp, agent, eps, TARGET_ACTION, targeted)

                    next_goal = agent.h(state, goal, next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(state, goal, next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    goal = next_goal

        episode_durations[eps].append(overall_reward / num_episodes)

In [14]:
noise_hrl = {'both': {}, 'action_only': {}, 'goal_only': {}, 'both_same': {}}
for l2norm in np.arange(0,0.51,0.05):
    for i in [noise_hrl['both'], noise_hrl['action_only'], noise_hrl['goal_only'], noise_hrl['both_same']]:
        i[np.round(l2norm, 2)] = []

targeted = {'goal': {}, 'action': {}}
untargeted = {'goal': {}, 'action': {}}
for eps in np.arange(0.0, 0.201, 0.02):
    for x in ['goal', 'action']:
        targeted[x][eps] = []
        untargeted[x][eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

#i = 1
#while i < 2:
#    #agent = train_model()
#    agent = HIRO(n_observations, n_actions).to(device)
#    load_model(agent, f"hiro_fall_{i}")
#
#    if agent is not None:
#        # goal_attack, action_attack, same_noise
#        eval_model(agent, noise_hrl['both_same'], True, True, True)
#        eval_model(agent, noise_hrl['both'], True, True, False)
#        eval_model(agent, noise_hrl['action_only'], False, True, False)
#        eval_model(agent, noise_hrl['goal_only'], True, False, False)
#        print(f"{i} noise_hrl: {noise_hrl}")
#        i += 1

#print("----")
#print(f"noise_hrl: {noise_hrl}")

untargeted = {'goal': {0.0: [], 0.02: [], 0.04: [], 0.06: [], 0.08: [], 0.1: [], 0.12: [], 0.14: [], 0.16: [], 0.18: [], 0.2: []}, 'action': {0.0: [90.71599999998378, 93.92799999998951, 89.32399999998182, 95.33199999999141], 0.02: [52.47600000000746, 10.763999999994896, 82.37599999997911, 28.44600000000618], 0.04: [10.68899999999834, -34.03999999997454, 61.0819999999891, -6.698000000004153], 0.06: [-31.369999999978273, -50.00000000000659, 11.175999999997046, -34.35099999997142], 0.08: [-34.8049999999718, -50.00000000000659, 18.711000000008205, -48.68500000000203], 0.1: [-32.447999999975025, -50.00000000000659, -9.309000000007101, -45.83499999999325], 0.12: [-26.271999999980235, -50.00000000000659, -0.5940000000016307, -50.00000000000659], 0.14: [-15.960999999980013, -50.00000000000659, -6.242000000002596, -47.6320000000007], 0.16: [-37.81399999996952, -50.00000000000659, -37.572999999968516, -48.58500000000071], 0.18: [-18.029999999985556, -50.00000000000659, -50.00000000000659, -50.00000000000659], 0.2: [-27.832999999978846, -50.00000000000659, -50.00000000000659, -48.61300000000063]}}
targeted = {'goal': {0.0: [], 0.02: [], 0.04: [], 0.06: [], 0.08: [], 0.1: [], 0.12: [], 0.14: [], 0.16: [], 0.18: [], 0.2: []}, 'action': {0.0: [90.45499999998498, 94.04099999998914, 89.78899999998244, 95.26399999999205], 0.02: [75.06599999998556, 81.1359999999862, 24.36700000001929, -29.621999999974456], 0.04: [-0.024000000004946287, 30.68100000002078, -33.10299999997487, -47.38099999999524], 0.06: [-14.272000000000704, -29.439999999974237, -45.985999999989254, -44.694999999985924], 0.08: [-10.083000000006804, -45.19499999999091, -48.98300000000652, -46.063999999991566], 0.1: [-7.620000000002157, -45.05599999998496, -50.00000000000659, -41.88299999997487], 0.12: [-13.934999999992876, -50.00000000000659, -47.49899999999567, -36.650999999972775], 0.14: [-36.20999999996887, -48.74400000000202, -44.87699999998544, -36.40399999996989], 0.16: [-40.647999999967844, -50.00000000000659, -43.43099999998813, -34.80999999997133], 0.18: [-47.58899999999713, -50.00000000000659, -43.98599999998016, -40.39799999996984], 0.2: [-43.62399999997998, -50.00000000000659, -42.28299999997391, -41.72699999997644]}}

i = 5
while i < 6:
    if i == 1:
        i += 1
        continue

    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_fall_{i}")

    if agent is not None:
        apply_fgsm(agent, untargeted['action'], False)   
        print(f"{i} fgsm (ut): {untargeted}")

        apply_fgsm(agent, targeted['action'], True)   
        print(f"{i} fgsm (t): {targeted}")
        i += 1

print("----")
print(f"fgsm (ut): {untargeted}")
print(f"fgsm (t): {targeted}")

5 fgsm (ut): {'goal': {0.0: [], 0.02: [], 0.04: [], 0.06: [], 0.08: [], 0.1: [], 0.12: [], 0.14: [], 0.16: [], 0.18: [], 0.2: []}, 'action': {0.0: [90.71599999998378, 93.92799999998951, 89.32399999998182, 95.33199999999141, 91.60099999998562], 0.02: [52.47600000000746, 10.763999999994896, 82.37599999997911, 28.44600000000618, 90.60499999998424], 0.04: [10.68899999999834, -34.03999999997454, 61.0819999999891, -6.698000000004153, 82.59999999997743], 0.06: [-31.369999999978273, -50.00000000000659, 11.175999999997046, -34.35099999997142, 58.56499999998063], 0.08: [-34.8049999999718, -50.00000000000659, 18.711000000008205, -48.68500000000203, -12.973999999984148], 0.1: [-32.447999999975025, -50.00000000000659, -9.309000000007101, -45.83499999999325, -23.29199999998142], 0.12: [-26.271999999980235, -50.00000000000659, -0.5940000000016307, -50.00000000000659, -30.55799999997628], 0.14: [-15.960999999980013, -50.00000000000659, -6.242000000002596, -47.6320000000007, -36.46699999996778], 0.16: 

In [ ]:
Solved after 2188 episodes!
0 noise_hrl: {'both': {0.0: [91.14999999998405], 0.05: [79.29499999998413], 0.1: [67.06299999997272], 0.15: [66.75099999998633], 0.2: [65.29299999997187], 0.25: [70.10999999997367], 0.3: [69.03899999996797], 0.35: [68.90299999997333], 0.4: [59.02899999998925], 0.45: [37.38900000002316], 0.5: [41.19000000002035]}, 'action_only': {0.0: [90.8299999999855], 0.05: [81.12999999998142], 0.1: [78.68399999997833], 0.15: [73.29799999997664], 0.2: [64.69999999998511], 0.25: [41.89100000001744], 0.3: [36.32200000001718], 0.35: [33.518000000017366], 0.4: [32.30900000001845], 0.45: [27.87500000002194], 0.5: [23.13200000001679]}, 'goal_only': {0.0: [90.50499999998341], 0.05: [79.91199999997998], 0.1: [71.54099999997774], 0.15: [50.45499999999405], 0.2: [52.255999999993335], 0.25: [64.18799999996844], 0.3: [61.16999999998588], 0.35: [62.519999999980975], 0.4: [54.405999999974576], 0.45: [50.605999999991944], 0.5: [40.07300000002481]}, 'both_same': {0.0: [91.3149999999862], 0.05: [77.30599999997263], 0.1: [57.20799999997914], 0.15: [57.40599999997899], 0.2: [73.60999999997127], 0.25: [62.46399999997322], 0.3: [64.08899999996703], 0.35: [50.193999999994595], 0.4: [28.695000000018705], 0.45: [21.46100000000863], 0.5: [20.46200000001309]}}
Solved after 566 episodes!
1 noise_hrl: {'both': {0.0: [76.48099999997548], 0.05: [66.42399999998551], 0.1: [74.93699999996724], 0.15: [36.782000000018904], 0.2: [26.905000000016084], 0.25: [3.232999999994894], 0.3: [1.715000000000258], 0.35: [-3.2350000000003205], 0.4: [-16.95199999999436], 0.45: [-9.302000000000056], 0.5: [-22.68099999997717]}, 'action_only': {0.0: [81.90599999998186], 0.05: [64.78699999998383], 0.1: [75.11099999997964], 0.15: [80.14799999997838], 0.2: [79.4489999999731], 0.25: [79.66899999998031], 0.3: [74.24299999997808], 0.35: [71.39599999997375], 0.4: [46.79400000000997], 0.45: [23.03300000001197], 0.5: [7.067999999995821]}, 'goal_only': {0.0: [72.13599999998291], 0.05: [81.40199999997735], 0.1: [75.3829999999819], 0.15: [47.44400000000806], 0.2: [35.46500000001251], 0.25: [11.798999999998209], 0.3: [-16.87699999998201], 0.35: [-24.64499999997189], 0.4: [-24.263999999976885], 0.45: [-31.794999999969306], 0.5: [-22.558999999977758]}, 'both_same': {0.0: [86.30399999998434], 0.05: [59.75999999999748], 0.1: [50.94400000000248], 0.15: [30.537000000021575], 0.2: [4.668999999996821], 0.25: [-9.064000000005079], 0.3: [-14.91999999999718], 0.35: [-27.673999999978474], 0.4: [-31.97899999997428], 0.45: [-33.903999999973045], 0.5: [-30.797999999973207]}}
Solved after 957 episodes!
Solved after 2026 episodes!
3 noise_hrl: {'both': {0.0: [94.2229999999902, 89.37699999998289], 0.05: [57.017999999999596, 75.04199999996837], 0.1: [-2.9510000000033165, 59.294999999971544], 0.15: [-30.59899999997835, 13.106000000013374], 0.2: [-45.90099999999438, -15.556999999989483], 0.25: [-45.89399999999006, -29.576999999975257], 0.3: [-41.75099999997314, -17.43499999998265], 0.35: [-46.385999999995256, -20.941999999986816], 0.4: [-48.65700000000079, -9.663000000009037], 0.45: [-43.94499999998115, -13.374999999997538], 0.5: [-46.23199999999129, -1.970000000002649]}, 'action_only': {0.0: [93.97199999998942, 89.64899999998318], 0.05: [80.29699999998728, 90.11699999998332], 0.1: [68.43499999997522, 82.77699999998451], 0.15: [58.758999999986266, 62.84499999998334], 0.2: [34.252000000016416, 35.68800000002029], 0.25: [-6.889000000004367, 18.44600000000853], 0.3: [-34.813999999977135, 3.469999999999179], 0.35: [-43.170999999978946, 6.354999999995261], 0.4: [-45.93799999998908, -12.834999999999079], 0.45: [-48.75600000000115, -22.37399999998165], 0.5: [-50.00000000000659, -27.589999999972427]}, 'goal_only': {0.0: [93.63399999998886, 89.6589999999833], 0.05: [76.67699999998851, 51.43299999999118], 0.1: [15.861000000010801, 50.308999999989176], 0.15: [-21.990999999979348, 10.792999999998555], 0.2: [-47.11599999999541, -17.394999999984194], 0.25: [-36.54499999996976, -36.60199999996983], 0.3: [-50.00000000000659, -43.01099999998002], 0.35: [-47.522999999995754, -39.2579999999682], 0.4: [-47.28700000000058, -36.076999999969274], 0.45: [-48.54500000000493, -34.72399999997301], 0.5: [-50.00000000000659, -27.237999999975838]}, 'both_same': {0.0: [93.92799999998986, 89.82199999998213], 0.05: [75.27099999998303, 68.19199999996806], 0.1: [9.72299999999606, 43.01200000002319], 0.15: [-31.905999999974874, 1.6719999999995085], 0.2: [-44.80399999998717, -23.92799999997686], 0.25: [-50.00000000000659, -34.14099999996831], 0.3: [-48.629000000005234, -28.20799999997092], 0.35: [-46.012999999990214, -30.569999999970136], 0.4: [-48.77000000000234, -29.166999999974017], 0.45: [-48.856000000001515, -16.66199999999904], 0.5: [-50.00000000000659, -12.425999999984592]}}
Solved after 784 episodes!
Solved after 2229 episodes!
5 noise_hrl: {'both': {0.0: [94.97999999999246, 91.5919999999854], 0.05: [56.68300000000006, 74.89799999997648], 0.1: [9.994999999994992, 22.664000000011555], 0.15: [-14.511999999985617, -6.061000000007647], 0.2: [-35.6279999999706, -10.002000000003678], 0.25: [-41.36299999996994, -33.816999999968374], 0.3: [-44.02999999998627, -30.34299999997835], 0.35: [-47.41599999999628, -28.75599999997639], 0.4: [-41.851999999973835, -29.12799999997753], 0.45: [-34.55699999997266, -23.937999999984427], 0.5: [-36.359999999973574, -33.53399999997518]}, 'action_only': {0.0: [95.28699999999212, 91.6059999999855], 0.05: [52.99099999997989, 90.55299999998476], 0.1: [25.192000000017558, 88.58299999998157], 0.15: [19.207000000007316, 81.90599999997528], 0.2: [-10.057999999997675, 63.180999999975626], 0.25: [-4.806000000003188, 54.20699999998302], 0.3: [0.2920000000004416, 35.14500000001777], 0.35: [-4.368999999999773, 29.91600000001266], 0.4: [-9.963999999994, 22.199000000007448], 0.45: [-12.001999999999645, 15.421000000003477], 0.5: [-14.597999999988918, 1.3589999999978093]}, 'goal_only': {0.0: [95.42999999999185, 91.60599999998561], 0.05: [79.05399999998485, 67.55199999997275], 0.1: [17.246000000001995, 28.090000000018918], 0.15: [0.997999999997677, -6.254000000005414], 0.2: [-17.184999999976814, -22.133999999982237], 0.25: [-39.11999999996896, -19.756999999983346], 0.3: [-37.3509999999699, -38.76399999996979], 0.35: [-34.30399999997176, -37.72999999997096], 0.4: [-36.83899999996836, -33.82299999996924], 0.45: [-42.673999999977134, -35.832999999972856], 0.5: [-38.47699999996971, -37.97599999997616]}, 'both_same': {0.0: [95.4979999999918, 91.51199999998536], 0.05: [57.62699999999806, 71.3169999999803], 0.1: [24.45200000001455, 23.49500000002157], 0.15: [-22.03299999997608, 0.49099999999980876], 0.2: [-16.436999999991862, -6.54800000000321], 0.25: [-38.18899999996912, -13.08199999999984], 0.3: [-41.67599999997583, -22.320999999988995], 0.35: [-42.86099999997785, -27.821999999984627], 0.4: [-38.64299999996989, -25.055999999976315], 0.45: [-44.16199999999643, -32.56099999996903], 0.5: [-42.28999999997421, -31.143999999971435]}}

In [ ]:
def eval_scale(agent, episode_durations):
    agent.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for scale in np.arange(1.0,7.01,0.5):
        env = NormalizedEnv(PointFallEnv(scale))

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal = agent.select_goal(g_state, False, False)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(scale, 2)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for scale in np.arange(1.0,7.01,0.5):
    episodes[np.round(scale, 2)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 6:
    if i == 1:
        i += 1
        continue
        
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_fall_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_scale(agent, episodes)
        print(f"{i} scale: {episodes}")
        i += 1

print("----")
print(f"scale: {episodes}")

0 scale: {1.0: [-40.22599999997028], 1.5: [-30.681999999974636], 2.0: [-22.282999999985723], 2.5: [14.147000000007887], 3.0: [79.62599999997866], 3.5: [90.61799999998436], 4.0: [91.16199999998582], 4.5: [90.0439999999843], 5.0: [88.90199999998147], 5.5: [84.26999999997392], 6.0: [84.96199999997742], 6.5: [85.80999999998143], 7.0: [83.16699999997368]}
1 scale: {1.0: [-40.22599999997028, -0.6030000000028386], 1.5: [-30.681999999974636, 65.05799999999346], 2.0: [-22.282999999985723, 75.27299999998534], 2.5: [14.147000000007887, 90.1799999999845], 3.0: [79.62599999997866, 84.17599999997967], 3.5: [90.61799999998436, 79.69799999997961], 4.0: [91.16199999998582, 73.58199999998902], 4.5: [90.0439999999843, 74.59699999999022], 5.0: [88.90199999998147, 82.30499999998256], 5.5: [84.26999999997392, 85.76599999997883], 6.0: [84.96199999997742, 85.01999999998006], 6.5: [85.80999999998143, 84.2189999999793], 7.0: [83.16699999997368, 86.61299999997884]}
2 scale: {1.0: [-40.22599999997028, -0.60300000

In [ ]:
def eval_starting_position(agent, episode_durations):
    agent.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for extra_range in np.arange(0.0, 0.401, 0.05):
        
        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            extra = np.random.uniform(-0.1 - extra_range, 0.1 + extra_range, env.starting_point.shape)
            #extra = np.random.uniform(0.1, 0.1 + extra_range, env.starting_point.shape)
            #extra = extra * (2*np.random.randint(0,2,size=env.starting_point.shape)-1)
            env.unwrapped.state = np.array(env.starting_point + extra, dtype=np.float32)
            env.unwrapped.state[2] += math.pi / 2. # start facing up
            env.unwrapped.state[2] = env.state[2] % (2 * math.pi)
            observation = env.normalised_state()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal = agent.select_goal(g_state, False, False)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(extra_range, 3)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for extra_range in np.arange(0.0, 0.401, 0.05):
    episodes[np.round(extra_range, 3)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

env = NormalizedEnv(PointFallEnv(4))
i = 0
while i < 6:
    if i == 1:
        i += 1
        continue
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_fall_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_starting_position(agent, episodes)
        print(f"{i} range: {episodes}")
        i += 1

print("----")
print(f"range: {episodes}")

0 range: {0.0: [91.0559999999851], 0.05: [91.52499999998567], 0.1: [91.23999999998425], 0.15: [91.29499999998501], 0.2: [88.57199999998473], 0.25: [88.52199999998429], 0.3: [84.70899999998248], 0.35: [90.07999999998634], 0.4: [80.52299999998537]}
1 range: {0.0: [91.0559999999851, 80.70299999997947], 0.05: [91.52499999998567, 80.75899999998009], 0.1: [91.23999999998425, 83.51999999998402], 0.15: [91.29499999998501, 79.53999999997903], 0.2: [88.57199999998473, 85.14599999998109], 0.25: [88.52199999998429, 75.16499999998868], 0.3: [84.70899999998248, 83.99999999998253], 0.35: [90.07999999998634, 68.18599999998632], 0.4: [80.52299999998537, 72.64399999998317]}
2 range: {0.0: [91.0559999999851, 80.70299999997947, 93.88899999998945], 0.05: [91.52499999998567, 80.75899999998009, 94.04799999998957], 0.1: [91.23999999998425, 83.51999999998402, 94.19099999998997], 0.15: [91.29499999998501, 79.53999999997903, 93.73299999998866], 0.2: [88.57199999998473, 85.14599999998109, 93.9079999999893], 0.25:

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def save_trajectories(agent, episode_durations, dirty):
    agent.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.controller.is_training = False

    num_episodes = 10

    c = 10

    l2norm = 0.3
    episode_durations.append([])
    
    for i_episode in range(num_episodes):
        path = {"overall_reward": 0, "manager": [], "worker": []}

        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        g_state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)

        if dirty:
            g_state = g_state + state_range * noise
            g_state = torch.max(torch.min(g_state, state_max), state_min).float()
        if dirty:
            state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
            state = torch.max(torch.min(state, state_max), state_min).float()

        episode_steps = 0
        overall_reward = 0
        done = False
        while not done:
            # select a goal
            goal = agent.select_goal(g_state, False, False)
            path["manager"].append((episode_steps, g_state_.detach().cpu().squeeze(0).numpy(), goal.detach().cpu().squeeze(0).numpy()))

            goal_done = False
            while not done and not goal_done:
                action = agent.select_action(state, goal, False, False)
                path["worker"].append((episode_steps, torch.cat([state_, goal], 1).detach().cpu().squeeze(0).numpy(), action.detach().cpu().squeeze(0).numpy()))
                observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                g_state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                if dirty:
                    g_next_state = g_next_state + state_range * noise
                    g_next_state = torch.max(torch.min(g_next_state, state_max), state_min).float()
                if dirty:
                    next_state = next_state + state_range * torch.FloatTensor(next_state.shape).uniform_(-l2norm, l2norm).to(device)
                    next_state = torch.max(torch.min(next_state, state_max), state_min).float()

                next_goal = agent.h(g_state, goal, g_next_state)
                                  
                overall_reward += reward

                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                episode_steps += 1

                #goal_done = agent.goal_reached(action, goal)
                goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                if (episode_steps % c) == 0:
                    goal_done = True

                state = next_state
                g_state = g_next_state
                goal = next_goal

        path["overall_reward"] = overall_reward
        episode_durations[-1].append(path)

In [ ]:
episodes = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 6:
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        save_trajectories(agent, episodes, True)
        #print(f"{i} paths: {episodes}")
        i += 1

print("----")
#print(f"paths: {episodes}")

torch.save(episodes, "PointFall_dirty_eps.pt")

----
